# Implanatação de algumas Bibliotacas

In [ ]:
import csv
import numpy as np
import pandas as pd
import sys
from tkinter import Tk
from tkinter import filedialog
from datetime import date, datetime, timedelta
#import os
#from ciso8601 import parse_datetime

# Criando variaveis para os arquivos permitidos

In [ ]:
p_cmpid13  = pd.read_excel('PQ409.xlsx')      ### leitura do arquivo com os sinais dos cpmids do quadro 409
param13    = pd.read_excel('LQ409.xlsx')      ### leitura do arquivo com o layout atualizado do quadro 409

In [ ]:
control  = 0                                              ### controle de backup    
p_cmpid13['cmpid'] = p_cmpid13.cmpid.astype(str)            ### transforma o campo cmpid - QE 409 - do arq auxiliar para string

In [ ]:
## FUNÇÃO PARA SELEÇÃO DO ARQUIVO

class qe_s:

    def __init__(self):
        self.df = {nome[s]: [] for s in range(nome.shape[0])}
  
    def make_df(self):
        lines = []
        root = Tk()
        root.attributes("-topmost", True)
        root.withdraw()
        for file in filedialog.askopenfilenames(filetypes=[("Arquivos de Text", "*.txt")]):
            for line in open(file, encoding="utf-8"):
                line = line.replace(",",".").strip()
                for x in range(nome.shape[0]):
                    self.df[nome[x]].append(line[i[x]:j[x]])
                    
        return pd.DataFrame(self.df)

In [ ]:
nome     = param13.Campo                      ### guarda os nomes das colunas existentes no quadro 409
i        = param13.Ind_inf                    ### guarda a posição no arquivo que inicia um determinado campo
j        = param13.Ind_sup                    ### guarda a posição no arquivo que termina um determinado campo

quadro = qe_s()       
df_main = quadro.make_df()
df_main

## SELECINE O(S) ARQUIVO(S) DESEJADO(S)

In [ ]:
###################################
###Importacao do Quadro409.xlsx ###
####que serão usados para fins#####
######### de comparação ###########
###################################

#Pasta ENTCODIGO
arq_entcod = pd.read_excel('Quadro409.xlsx','Cod_SUSEP',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta MRFMESANO
arq_mrfmesano = pd.read_excel('Quadro409.xlsx','MRFMESANO',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta RAMCODIGO
arq_ramcod = pd.read_excel('Quadro409.xlsx','RAMCODIGO',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta TPMORESSID
arq_tpmoressid = pd.read_excel('Quadro409.xlsx','TPMORESSID',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta MPRTIPOCONT
arq_mprtipocont = pd.read_excel('Quadro409.xlsx','MPRTIPOCONT',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta MPRMODCONT
arq_mprmodcont = pd.read_excel('Quadro409.xlsx','MPRMODCONT',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta MPRBASEIND
arq_mprbaseind = pd.read_excel('Quadro409.xlsx','MPRBASEIND',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta MPRMOEDA
arq_mprmoeda = pd.read_excel('Quadro409.xlsx','MPRMOEDA',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

In [ ]:
# Criando uma lista para guardar todas as criticas impeditivas
criticas_impeditivas = []
criticas = []

# Criticas (Validacoes)

In [ ]:
#8048.2 Verifica o tamanho padrão da linha (deve conter 172 caracteres)
tamanho = 0
for i in range(0,df_main.shape[0]):                    #Passando por cada linha
    for iten in df_main.columns:                       #Passando por cada coluna
        tamanho = tamanho + len(df_main[iten][i])      #Contando Caracteres por linha
    if tamanho != 172:
        print('Erro na linha ',i,' o número de caracteres é de: ',tamanho)
        critica8048_2 = ("Critica 8048.2, Erro na linha ",i," o número de caracteres é de: ",tamanho)
        criticas_impeditivas.append(critica8048_2)
    tamanho = 0

In [ ]:
#8048.3 Verifica se o campo sequencial MPRSEQ é uma sequência válida, que se inicia em 0000001
array3 = df_main['MPRSEQ']
x = list(array3)                                     ##Dados do arquivo
arrayteste = x[:]
arrayteste.sort()                                    ##Dados sequenciados
if (arrayteste == x):                                ##Comparando
    print('Tudo certo, esta de forma sequencial.')
else: 
    print('ERRO!!!, Não prossiga.')
    critica8048_3 = str('Critica 8048.3, Não temos uma sequencia valida na coluna MPRSEQ')
    criticas_impeditivas.append(critica8048_3)

In [ ]:
#8048.4 Verifica se o campo ENTCODIGO corresponde à sociedade que está enviando o FIP/SUSEP
entcod = set(arq_entcod['Cod_SUSEP'].astype('str'))                   #Escolhendo a coluna do arquivo modelo
dadosentcod = set(df_main['ENTCODIGO'])                               #Escolhendo a coluna dos dados
for itemtentcod in dadosentcod: 
    if itemtentcod not in entcod:
        print('ERRO!! ENTCODIGO não corresponde a uma operação valida', itemtentcod)
        critica8048_4 = ('Critica 8048.4, ENTCODIGO não corresponde a uma operação valida', itemtentcod)
        criticas_impeditivas.append(critica8048_4)
    else:
        print('ENTCODIGO corresponde a um tipo de operação válida.', itemtentcod)
        continue

In [ ]:
#8048.5 Verifica se o campo MRFMESANO corresponde a um ano e mês válidos
try:
    mr = (df_main['MRFMESANO'] + '01')
    mr = pd.to_datetime(mr)
    print(mr)
except ValueError:
    print('ERROOO!! MRFMESANO corresponde a um ano e mês válidos')
    critica8048_5 = ('Critica 8048.5, MRFMESANO corresponde a um ano e mês válidos')
    criticas_impeditivas.append(critica8048_5)

In [ ]:
#8048.6 Verifica se o campo TPMORESSID corresponde a um tipo de movimento válido (conforme tabela 'TiposMovimentosResseguros' do FIPSUSEP) 
tpmoressid = set(arq_tpmoressid['TPMORESSID'].astype('int64'))                 #Escolhendo a coluna do arquivo modelo
dadostpmoressid = set(df_main['TPMORESSID'].astype('int64'))                                 #Escolhendo a coluna dos dados
for itemtpmoressid in dadostpmoressid: 
    if itemtpmoressid not in tpmoressid:
        print('ERRO!! TPMORESSID não corresponde a um tipo de movimento válido (conforme tabela TiposMovimentosResseguros do FIPSUSEP)', itemtpmoressid)
        critica8048_6 = ('Critica 8048.6, TPMORESSID não corresponde a um tipo de movimento válido (conforme tabela TiposMovimentosResseguros do FIPSUSEP)', itemtpmoressid)
        criticas_impeditivas.append(critica8048_6)
    else:
        print('TPMORESSID corresponde a um tipo de movimento válido (conforme tabela TiposMovimentosResseguros do FIPSUSEP)', itemtpmoressid)
        continue

In [ ]:
#8048.7 Verifica se o campo GRACODIGO corresponde a um grupo de ramos válido operado pelo ressegurador 
ramcod = set(arq_ramcod['graCodigo'].astype('str'))                 #Escolhendo a coluna do arquivo modelo
dadosramcod = set(df_main['GRACODIGO'])                                 #Escolhendo a coluna dos dados
for itemramcod in dadosramcod: 
    if itemramcod not in ramcod:
        print('ERRO!! GRACODIGO não corresponde a um grupo de ramos válido operado pelo ressegurador', itemramcod)
        critica8048_7 = ('Critica 8048.7, GRACODIGO não corresponde a um grupo de ramos válido operado pelo ressegurador', itemramcod)
        criticas_impeditivas.append(critica8048_7)
    else:
        print('GRACODIGO corresponde a um grupo de ramos válido operado pelo ressegurador', itemramcod)
        continue

In [ ]:
#8048.9 Verifica se o campo MPRTIPOCONT foi preenchido com um tipo de contrato válido
mprtipocont = set(arq_mprtipocont['MPRTIPOCONT'].astype('str'))                 #Escolhendo a coluna do arquivo modelo
dadosmprtipocont = set(df_main['MPRTIPOCONT'])                                 #Escolhendo a coluna dos dados
for itemmprtipocont in dadosmprtipocont: 
    if itemmprtipocont not in mprtipocont:
        print('ERRO!! MPRTIPOCONT não foi preenchido com um tipo de contrato válido', itemmprtipocont)
        critica8048_9 = ('Critica 8048.9, MPRTIPOCONT não foi preenchido com um tipo de contrato válido', itemmprtipocont)
        criticas_impeditivas.append(critica8048_9)
    else:
        print('MPRTIPOCONT foi preenchido com um tipo de contrato válido', itemmprtipocont)
        continue

In [ ]:
#8048.10 Verifica se o campo MPRMODCONT foi preenchido com uma modalidade de contrato válida, exceto nos casos em que o tipo de contrato seja ‘Facultativo’, quando o campo deve ser preenchido com ‘99’ 
mprmodcont = set(arq_mprmodcont['MPRMODCONT'].astype('int64'))                 #Escolhendo a coluna do arquivo modelo
dadosmprmodcont = set(df_main['MPRMODCONT'].astype('int64'))                                 #Escolhendo a coluna dos dados
for itemmprmodcont in dadosmprmodcont: 
    if itemmprmodcont not in mprmodcont:
        print('ERRO!! MPRMODCONT não foi preenchido com uma modalidade de contrato válido', itemmprmodcont)
        critica8048_10 = ('Critica 8048.10, MPRMODCONT não foi preenchido com uma modalidade de contrato válido', itemmprmodcont)
        criticas_impeditivas.append(critica8048_10)
    else:
        print('MPRMODCONT foi preenchido com uma modalidade de contrato válido', itemmprmodcont)
        continue

In [ ]:
#8048.11 Para o tipo de contrato 'Facultativo', verifica se o tipo de movimento é 'Emissão de Prêmio Efetivo'35 ou 'Endosso de Prêmio Efetivo'36 ou 'Restituição de Prêmio Efetivo'37 ou 'Cancelamento de Prêmio Efetivo'38 ou 'Informação sem Movimentação de Prêmio'45 
mask11_tipocont = (df_main['MPRTIPOCONT'] == '2')
mask11_tpmoresid = ((df_main['TPMORESSID'] != '035') & (df_main['TPMORESSID'] != '036') & (df_main['TPMORESSID'] != '037') & (df_main['TPMORESSID'] != '038') & (df_main['TPMORESSID'] != '045'))
mask11 = (mask11_tipocont) & (mask11_tpmoresid)
df11 = df_main.loc[mask11]
if len(df11) > 0:
    df11.loc[:,'ID da Critica'] = ('CRITICA 8048.11')
df11

In [ ]:
#8048.12 Para o tipo de contrato 'Automático' e para as modalidades de contrato 'Proporcional', verifica se o tipo de movimento é 'Emissão de Prêmio Efetivo' ou 'Endosso de Prêmio Efetivo' ou 'Restituição de Prêmio Efetivo' ou 'Cancelamento de Prêmio Efetivo' ou 'Ajuste de Prêmio Efetivo' ou 'Emissão de Prêmio Estimado' ou 'Alteração de Prêmio Estimado' ou 'Cancelamento de Prêmio Estimado' ou 'Ajuste de Prêmio Estimado' ou 'Informação sem Movimentação de Prêmio' 
mask12_tipocont = ((df_main['MPRTIPOCONT'] == '1') & ((df_main['MPRMODCONT'] == '01')|(df_main['MPRMODCONT'] == '02')))
mask12_tpmoresid = (df_main['TPMORESSID'] == '044')
mask12 = (mask12_tipocont) & (mask12_tpmoresid)
df12 = df_main.loc[mask12]
if len(df12) > 0:
    df12.loc[:,'ID da Critica'] = ('CRITICA 8048.12')
df12

In [ ]:
#8048.13 Para o tipo de contrato 'Automático' e para as modalidades de contrato 'Não Proporcional' e 'Clash', verifica se o tipo de movimento é 'Emissão de Prêmio Efetivo' ou 'Endosso de Prêmio Efetivo' ou 'Restituição de Prêmio Efetivo' ou 'Cancelamento de Prêmio Efetivo' ou ' Ajuste de Prêmio Efetivo' ou 'Prêmio de Reintegração' ou 'Informação sem Movimentação de Prêmio' 
mask13_tipocont = ((df_main['MPRTIPOCONT'] == '1') & ((df_main['MPRMODCONT'] == '03')|(df_main['MPRMODCONT'] == '04')|(df_main['MPRMODCONT'] == '05')|(df_main['MPRMODCONT'] == '06')))
mask13_tpmoresid = ((df_main['TPMORESSID'] == '035') & (df_main['TPMORESSID'] == '036') & (df_main['TPMORESSID'] == '037') & (df_main['TPMORESSID'] == '038'))
mask13 = (mask13_tipocont) & (mask13_tpmoresid)
df13 = df_main.loc[mask13]
if len(df13) > 0:
    df13.loc[:,'ID da Critica'] = ('CRITICA 8048.13')
df13

In [ ]:
#8048.14 Verifica se os campos MPRDATACEITE, MPRDATAINICIO, MPRDATAFIM e MPRDATAEMISS correspondem a uma data válida 

#####################################################
##Readicionando as colunas de Data de str para date##
#####################################################

df_main.loc[:,'DTMPRDATACEITE'] = df_main['MPRDATACEITE'].astype('datetime64')
df_main.loc[:,'DTMPRDATAINICIO'] = df_main['MPRDATAINICIO'].astype('datetime64')
df_main.loc[:,'DTMPRDATAFIM'] = df_main['MPRDATAFIM'].astype('datetime64')
df_main.loc[:,'DTMPRDATAEMISS'] = df_main['MPRDATAEMISS'].astype('datetime64')

In [ ]:
#8048.15 Verifica se o campo MPRDATACONTR foi preenchido com uma data válida ou com '99999999'
mask15 = set(df_main['MPRDATACONTR'])
for item15 in mask15:
    try:
        x = pd.to_datetime(item15)
        print('Valida ',x)
    except ValueError:
        if item15 == '99999999':
            print("Valida Campo preenchido com ",item15)
        else:
            print("ERROO!!! Campo preenchido com ",item15)
            critica8048_15 = ('Critica 8048.15, MPRDATACONTR não correspondem a uma data válida', item15)
            criticas_impeditivas.append(critica8048_15)

In [ ]:
#8048.16 Verifica se o valor dos campos MPRVALORMOV, MPRVALORMOVCOMISS, MPRVALORMOVCORRET e MPRTAXACONV é float 

###########################################
##Readicionando as colunas str para float##
###########################################

df_main.loc[:,'fMPRVALORMOV'] = df_main['MPRVALORMOV'].astype('float64')
df_main.loc[:,'fMPRVALORMOVCOMIS'] = df_main['MPRVALORMOVCOMIS'].astype('float64')
df_main.loc[:,'fMPRVALORMOVCORRET'] = df_main['MPRVALORMOVCORRET'].astype('float64')
df_main.loc[:,'fMPRTAXACONV'] = df_main['MPRTAXACONV'].astype('float64')

#Adicionado colunas MPAVALORMOV, MPAVALORMOVCOMIS, MPAVALORMOVCORRET e MPATAXACONV no tipo Float

In [ ]:
#8048.17 Verifica se o campo MPRPERCRISCO corresponde a um valor entre '000,01' e '100,00' 
mask17 = ((df_main['MPRPERCRISCO'].astype('float64')) > 100.00) | ((df_main['MPRPERCRISCO'].astype('float64')) < 0.01)
df17 = df_main.loc[mask17]
if len(df17) > 0:
    df17.loc[:,'ID da Critica'] = ('CRITICA 8048.17')
df17

In [ ]:
#8048.18 Verifica se o campo MPRCODCORRET corresponde a um código de corretora de resseguro válido  ou '99999' 
mask18 = (((df_main['MPRCODCORRET'].astype('int64')) > 79999) | ((df_main['MPRCODCORRET'].astype('int64')) < 70000)) & ((df_main['MPRCODCORRET'].astype('int64')) != 99999)
df18 = df_main.loc[mask18]
if len(df18) > 0:
    df18.loc[:,'ID da Critica'] = ('CRITICA 8048.18')
df18

In [ ]:
#8048.19 Verifica se o campo MPRVIGMED corresponde a um valor entre '01' e '99' 
mask19 = ((df_main['MPRVIGMED'].astype('int64')) > 99) | ((df_main['MPRVIGMED'].astype('int64')) < 1)
df19 = df_main.loc[mask19]
if len(df19) > 0:
    df19.loc[:,'ID da Critica'] = ('CRITICA 8048.19')
df19

In [ ]:
#8048.20 Verifica se o campo MPRBASEIND foi preenchido com uma base indenitária válida 
mprbaseind = set(arq_mprbaseind['MPRBASEIND'].astype('int64'))                 #Escolhendo a coluna do arquivo modelo
dadosmprbaseind = set(df_main['MPRBASEIND'].astype('int64'))                                 #Escolhendo a coluna dos dados
for itemmprbaseind in dadosmprbaseind: 
    if itemmprbaseind not in mprbaseind:
        print('ERRO!! MPRBASEIND não foi preenchido com uma base indenitária válida', itemmprbaseind)
        critica8047_21 = ('Critica 8048.20, MPRBASEIND não foi preenchido com uma base indenitária válida', itemmprbaseind)
        criticas_impeditivas.append(critica8047_21)
    else:
        print('MPRBASEIND foi preenchido com uma base indenitária válida', itemmprbaseind)
        continue

In [ ]:
#8048.21 Verifica se o campo MPRMOEDA foi preenchido com uma moeda válida 
mprmoeda = set(arq_mprmoeda['Código'].astype('int64'))                 #Escolhendo a coluna do arquivo modelo
dadosmprmoeda = set(df_main['MPRMOEDA'].astype('int64'))                                 #Escolhendo a coluna dos dados
for itemmprmoeda in dadosmprmoeda: 
    if itemmprmoeda not in mprmoeda:
        print('ERRO!! MPRMOEDA não foi preenchido com uma moeda válida', itemmprmoeda)
        critica8048_21 = ('Critica 8048.21, MPRMOEDA não foi preenchido com uma moeda válida', itemmprmoeda)
        criticas_impeditivas.append(critica8048_21)
    else:
        print('MPRMOEDA foi preenchido com uma moeda válida', itemmprmoeda)
        continue

In [ ]:
criticas_impeditivas  #Lista das criticas impeditivas

In [ ]:
df_criticas_i = pd.DataFrame(criticas_impeditivas)     #criacao do DataFrame das criticas impeditivas
df_criticas_i = df_criticas_i.rename(columns={0:'Descrição'})
df_criticas_i

In [ ]:
df_criticas_i2 = pd.concat([df11,df12,df13,df17,df18,df19])
df_criticas_i2

In [ ]:
if criticas_impeditivas == []:                             # Verificando se a lista de criticas esta vazia
    print('Nenhuma Critica Impeditiva encontrada no Quadro 409')
else:
    print('Foi encontrada uma ou mais Criticas Impeditivas no Quadro 409')
    df_criticas_i.to_csv('Criticas_Impeditivas_409.csv')   # Gerando csv das criticas impeditivas

In [ ]:
if len(df_criticas_i2) == 0:                             # Verificando se o DataFrame de criticas esta vazia
    print('Nenhuma Critica Impeditiva encontrada no Quadro 409')
else:
    print('Foi encontrada uma ou mais Criticas Impeditivas no Quadro 409')
    df_criticas_i2.to_csv('Criticas_Impeditivas2_409.csv')   # Gerando csv das criticas impeditivas

# Criticas (Validacoes) Não Impeditivas

In [ ]:
#8048.8 Verifica se o campo MPRCODCESS corresponde a um código de sociedade válido
df_mains = df_main.copy()
df_mains['MPRCODCESS'] = df_mains['MPRCODCESS12'].astype('int64')
mask8 = ((df_mains['MPRCODCESS'].values < 1) & (df_mains['MPRCODCESS'].values > 19999)) | ((df_mains['MPRCODCESS'].values < 30000) & (df_mains['MPRCODCESS'].values > 59999))
df8 = df_main.loc[mask8]
if len(df8) > 0:
    df8.loc[:,'ID da Critica'] = ('CRITICA 8048.8')
df8

In [ ]:
#8048.22 Verifica se o valor do campo MPRVALORMOV é maior do que zero, exceto para os tipos de movimento 'Repasse de Ajuste de Prêmio - Prêmio Efetivo'39 ou 'Repasse de Ajuste de Prêmio - Prêmio Estimado'43 ou 'Alteração de Prêmio Estimado'41, quando o campo também pode assumir valor negativo, e para o tipo de movimento 'Informação sem Movimentação de Prêmio'45, quando o campo deve ser igual a zero. Quando o campo MPRVALORMOVCORRET for diferente de zero e o campo MPRVALORMOV for igual a zero, a regra não se aplica. 
mask22_1 = ((df_main['fMPRVALORMOV'] <= 0.0) & ((df_main['TPMORESSID'] != '039') & (df_main['TPMORESSID'] != '041') & (df_main['TPMORESSID'] != '043')  & (df_main['TPMORESSID'] != '045'))) & ((df_main['fMPRVALORMOVCORRET'] == 0.0) & (df_main['fMPRVALORMOV'] != 0.0))
df22_1 = df_main.loc[mask22_1]
if len(df22_1) > 0:
    df22_1.loc[:,'ID da Critica'] = ('CRITICA 8048.22')
df22_1

In [ ]:
mask22_2 = (df_main['fMPRVALORMOV'] != 0.0) & (df_main['TPMORESSID'] == '045')
df22_2 = df_main.loc[mask22_2]
if len(df22_2) > 0:
    df22_2.loc[:,'ID da Critica'] = ('CRITICA 8048.22')
df22_2

In [ ]:
#8048.23 Verifica se o valor do campo MPRVALORMOVCOMIS é maior ou igual a zero, exceto para o tipo de movimento ‘Prêmio de Reintegração’44 e 'Informação sem Movimentação de Prêmio'45 e para as modalidades de contrato 'Não Proporcional' e 'Clash', quando o valor do campo deve ser igual a zero; e para o tipo 'Ajuste de Prêmio Efetivo'39 ou 'Alteração de Prêmio Estimado'41 ou 'Repasse de Ajuste de Prêmio Estimado'43, quando o valor do campo pode assumir tanto valores positivos quanto valores negativos ou iguais a zero. 
mask23_1 = ((df_main['MPRVALORMOVCOMIS'].astype('float64')) < 0.0) & ((df_main['TPMORESSID'] != '039') & (df_main['TPMORESSID'] != '041') & (df_main['TPMORESSID'] != '043')) 
df23_1 = df_main.loc[mask23_1]
if len(df23_1) > 0:
    df23_1.loc[:,'ID da Critica'] = ('CRITICA 8048.23')
df23_1

In [ ]:
mask23_2 = ((df_main['MPRVALORMOVCOMIS'].astype('float64')) != 0.0) & ((df_main['TPMORESSID'] == '044') | (df_main['TPMORESSID'] == '045') | (df_main['MPRMODCONT'] == '03') | (df_main['MPRMODCONT'] == '04') | (df_main['MPRMODCONT'] == '05') | (df_main['MPRMODCONT'] == '06'))
df23_2 = df_main.loc[mask23_2]
if len(df23_2) > 0:
    df23_2.loc[:,'ID da Critica'] = ('CRITICA 8048.23')
df23_2

In [ ]:
#8048.25 Verifica para qualquer tipo de movimento, exceto pelo tipo de movimento ‘Informação sem Movimentação de Prêmio’, quando a modalidade do contrato for não proporcional, clash ou facultativo, se o mês e ano da data do campo MPRDATAEMISS é igual ao mês e ano da data do campo MRFMESANO.
mask25_rel = (df_main['TPMORESSID'] != '045') & (((df_main['MPRMODCONT'] == '99')|(df_main['MPRMODCONT'] == '06')|(df_main['MPRMODCONT'] == '03')|(df_main['MPRMODCONT'] == '04')|(df_main['MPRMODCONT'] == '05')))
maskmr = mr.dt.strftime("%Y-%m")
maskdtemi = (df_main['MPRDATAEMISS'].astype('datetime64')).dt.strftime("%Y-%m")

mask25 = ((maskmr != maskdtemi)) & (mask25_rel)
df25 = df_main.loc[mask25]
if len(df25) > 0:
    df25.loc[:,'ID da Critica'] = ('CRITICA 8048.25')
df25

In [ ]:
#8048.26 Verifica se a data de formalização do contrato (campo MPRDATACONTR) não ultrapassou 270 dias da data de início de vigência da cobertura (campo MPRDATAINICIO)
mask26_1 = (df_main['MPRDATACONTR'] != '99999999')
df26_1 =  df_main.loc[mask26_1]
for item26_1 in (df26_1['MPRDATACONTR']):
    try:
        v1 = pd.to_datetime(item26_1)
        print('Valida ',v1)
    except ValueError:
        if item26_1 == '99999999':
            print("Valida Campo preenchido com ",item26_1)
        else:
            print("Campo preenchido com ",item26_1)
            critica8048_26 = ('Critica 8048.26, MPRDATACONTR não correspondem a uma data válida', item26_1)
            criticas_impeditivas.append(critica8048_26)

In [ ]:
mask26_2 = (df_main['MPRDATAINICIO'] != '99999999')
df26_2 =  df_main.loc[mask26_2]

for item26_2 in (df26_2['MPRDATAINICIO']):
    try:
        v2 = pd.to_datetime(item26_2)
        print('Valida ',v2)
    except ValueError:
        if item26_2 == '99999999':
            print("Valida Campo preenchido com ",item26_2)
        else:
            print("Campo preenchido com ",item26_2)
            critica8048_26 = ('Critica 8048.26, MPRDATACONTR não correspondem a uma data válida', item26_2)
            criticas_impeditivas.append(critica8048_26)

In [ ]:
comparacao = (v1 + timedelta(days = 270)) < (v2)
if comparacao == True:
    print('ERRO!! A diferença de dias entre a data dos campos MPRDATACONTR e MPRDATAINICIO  ultrapassou 270 dias da data de início de vigência da cobertura')
    critica8048_26 = ('Critica 8048.26, A diferença de dias entre a data dos campos MPRDATACONTR e MPRDATAINICIO  ultrapassou 270 dias da data de início de vigência da cobertura')
    criticas.append(critica8048_26)
else:
    print('A diferença de dias entre a data dos campos MPRDATACONTR e MPRDATAINICIO não ultrapassou 270 dias da data de início de vigência da cobertura.')

In [ ]:
v1

In [ ]:
#8048.27 Para cada grupo de ramos, compara os valores registrados no campo MPRVALORMOV com os valores informados no Quadro 02R do FIPSUSEP (tolerância: R$ 1.000,00 ou 1% do valor informado no respectivo campo do Quadro 02R, o que for menor)

In [ ]:
#8048.28  Verifica se o campo MPRCODCESS não foi preenchido com '99999'
mask28 = (df_main['MPRCODCESS12'] == '99999')
df28 = df_main.loc[mask28]
if len(df28) > 0:
    df28.loc[:,'ID da Critica'] = ('CRITICA 8048.28')
df28

In [ ]:
################################################
# Adicionando Nome do Criador do output e data #
################################################

df_main.loc[:,'Criador'] = input("Insira seu nome: ") ### insere a coluna com o nome do criador da tabela
data_presente = datetime.now()
df_main.loc[:,'Data do output'] = data_presente ### insere a coluna com a data de criacao da tabela

In [ ]:
df_main

In [ ]:
criticas

In [ ]:
df_criticas = pd.DataFreame(criticas)
df_criticas = df_criticas.rename(columns={0:'Descrição'})
df_criticas

In [ ]:
df_criticas_2 = pd.concat([df8,df22_1,df22_2,df23_1,df23_2,df25,df28])
df_criticas_2

In [ ]:
if criticas == []:                   # Verificando se a lista de criticas esta vazia
    print('Nenhuma Critica NÃO Impeditiva encontrada no Quadro 409')
else:
    print('Foi encontrada uma ou mais Criticas NÃO Impeditivas no Quadro 409')
    df_criticas.to_csv('Criticas_409.csv')     # Gerando csv das criticas impeditivas

In [ ]:
if len(df_criticas_2) == 0:                             # Verificando se o DataFrame de criticas esta vazia
    print('Nenhuma Critica NÃO Impeditiva encontrada no Quadro 409')
else:
    print('Foi encontrada uma ou mais Criticas NÃO Impeditivas no Quadro 409')
    df_criticas_2.to_csv('Criticas_409_2.csv')         # Gerando csv das criticas impeditivas

In [ ]:
df_main.to_csv('409_Trabalho.csv')

# Teste

In [ ]:
# 5
mr = (df_main['MRFMESANO'] + '01')
mr = pd.to_datetime(mr)
maskmr = mr.dt.strftime("%Y-%m")


In [ ]:
x = (df_main['MRFMESANO'] + '01')
x = pd.to_datetime(x)
maskmr = x.dt.strftime("%Y-%m")
'''
maskreg = df_main['DTEBIDATAREG'].dt.strftime("%Y-%m")

mask26_1 = ((maskmr != maskreg)) & (mask26_rel)
df26_1 = df_main.loc[mask26_1]
if len(df26_1) > 0:
    df26_1.loc[:,'ID da Critica'] = ('CRITICA 7395.26')
df26_1
'''

In [ ]:
mr = (df_main['MRFMESANO'] + '01')
mr = pd.to_datetime(mr)
maskmr = mr.dt.strftime("%Y-%m")
maskmr

In [ ]:
try:
    mr = (df_main['MRFMESANO'] + '01')
    mr = pd.to_datetime(mr)
    print(mr)
except ValueError:
    print('ERROOO!! MRFMESANO corresponde a um ano e mês válidos')
    critica8048_5 = ('Critica 8048.5, MRFMESANO corresponde a um ano e mês válidos')
    criticas_impeditivas.append(critica8048_5)

In [ ]:
#8048.25 Verifica para qualquer tipo de movimento, exceto pelo tipo de movimento ‘Informação sem Movimentação de Prêmio’, quando a modalidade do contrato for não proporcional, clash ou facultativo, se o mês e ano da data do campo MPRDATAEMISS é igual ao mês e ano da data do campo MRFMESANO.

In [ ]:
mask26_rel = (df_main['TPMORESSID'] != '045') & (((df_main['MPRMODCONT'] == '99')|(df_main['MPRMODCONT'] == '06')|(df_main['MPRMODCONT'] == '03')|(df_main['MPRMODCONT'] == '04')|(df_main['MPRMODCONT'] == '05')))
maskmr = mr.dt.strftime("%Y-%m")
maskdtemi = (df_main['MPRDATAEMISS'].astype('datetime64')).dt.strftime("%Y-%m")

mask26 = ((maskmr != maskdtemi)) & (mask26_rel)
df26 = df_main.loc[mask26]
if len(df26) > 0:
    df26.loc[:,'ID da Critica'] = ('CRITICA 8047.26')
df26

In [ ]:
w = df_main['MPRDATAEMISS']
pd.to_datetime(w)

In [ ]:
maskmr = mr.dt.strftime("%Y-%m")
maskdtemi = df_main['MPRDATAEMISS'].dt.strftime("%Y-%m")

In [ ]:
#8048.26 Verifica se a data de formalização do contrato (campo MPRDATACONTR) não ultrapassou 270 dias da data de início de vigência da cobertura (campo MPRDATAINICIO)
mask26_1 = ((df_main['MPRDATACONTR'] != '99999999') & (df_main['MPRDATAINICIO'] != '99999999'))
df26_1 =  df_main.loc[mask26_1]
df26_1['MPRDATACONTR'] = (df26_1['MPRDATACONTR'].astype('datetime64'))
df26_1['MPRDATAINICIO'] = (df26_1['MPRDATAINICIO'].astype('datetime64'))
mask26 = (df26_1['MPRDATACONTR'] + timedelta(days = 270)) < (df26_1['MPRDATAINICIO'])
try:
    df26 = df_main.loc[mask26]
    if len(df26) > 0:
        df26.loc[:,'ID da Critica'] = ('CRITICA 8048.26')
    df26
except IndexingError:
    print('Tudo certo')